[![Topic Modeling in Python](https://img.youtube.com/vi/N0crN8YnF8Y/0.jpg)](https://www.youtube.com/watch?v=N0crN8YnF8Y&list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX)

# Imports 1
import os
import re
import requests
import shutil
import pandas as pd
from PIL import Image
import pytesseract
import time
import numpy as np
from pprint import pprint

In [2]:
# Imports 2
import os
from os.path import isfile, join
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import datetime

In [3]:
# Import 3 - Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# Import spacy for lemmatization
import spacy
from gensim.models.wrappers import LdaMallet
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'}) 

#You should update this path as per the path of Mallet directory on your system.
new_mallet_path = r'C:/new_mallet/mallet-2.0.8/bin/mallet'

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(sentence))

### Steps of the process

1. load the relevant datafarme
2. Takes the column
3. Runs the relevant opertions on the column


In [ ]:
tokens_20_cs_cf = list(sent_to_words(i2t_cleaned_20_cs_cf))

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Merl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words_nltk = stopwords.words('english')

# SKLEARN Stop Words
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
# stop_words_sklearn = list(ENGLISH_STOP_WORDS)

def remove_stopwords(texts, csw_list=None, csw_stopwords=False):
    
    #append the relevant custom stopword list to the standard stop word list
    if csw_stopwords == True:
        stop_words = stop_words_nltk + csw_list
    else:
        stop_words = stop_words_nltk
        
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
def make_bigrams(tokens, bigram):
    
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    return [bigram_mod[doc] for doc in tokens]

def make_trigrams(tokens, trigram, bigram):
    
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    return [trigram_mod[bigram_mod[doc]] for doc in tokens]

In [ ]:
def lemmatization(tokens, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tokens:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
tokens_nostops_20_cs_cf = remove_stopwords(tokens_20_cs_cf, csw_20_cs_cf, csw_stopwords=True)

tokens_bigrams_20_cs_cf = make_bigrams(tokens_nostops_20_cs_cf, bigram_20_cs_cf)

###### Lemmatization keeping only noun, adj, vb, adv

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

tokens_lemmatized_20_cs_cf = lemmatization(tokens_bigrams_20_cs_cf)



In [ ]:
# Create Dictionary
id2word_20_cs_cf = corpora.Dictionary(tokens_lemmatized_20_cs_cf)

# Create Corpus (a list of documents)
texts_20_cs_cf = tokens_lemmatized_20_cs_cf

# Term Document Frequency
corpus_20_cs_cf = [id2word_20_cs_cf.doc2bow(text) for text in texts_20_cs_cf]


# Build LDA model
lda_model_20_cs_cf = gensim.models.ldamodel.LdaModel(corpus=corpus_20_cs_cf,
                                           id2word=id2word_20_cs_cf,
                                           num_topics=25, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
#Print the Keyword in the 20 topics
pprint(lda_model_20_cs_cf.print_topics())

In [ ]:
#Function to alculate coherence score
def get_coherence(model, texts, dictionary, coherence="c_v"):
    
    #coherence score
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_c
    oherence()
    
    return coherence

In [ ]:
# Calculate coherence score
coherence_score_lda_20_cs_cf = coherence_model_lda_20_cs_cf.get_coherence()
print('\nCoherence Score: ', coherence_score_lda_20_cs_cf)